# Load GeoQuery Dataset

### GeoQuery dataset property:
1. logic forms are different (probably could rewrite ourselves)
2. We have could parse to get the query
3. Table schema we have


### Following is a description of the geobase predicates:

state(name, abbreviation, capital, population, area, state_number, city1, city2, city3, city4)

city(state, state_abbreviation, name, population)

river(name, length, [states through which it flows])

border(state, state_abbreviation, [states that border it])

highlow(state, state_abbreviation, highest_point, highest_elevation, lowest_point, lowest_elevation)

mountain(state, state_abbreviation, name, height)

road(number, [states it passes through])

lake(name, area, [states it is in])


### We can change to the format we want:

state, abbreviation, capital, population, area, state_number, cities, border, highest_point, highest_elevation, lowest_point, lowest_elevation, rivers, mountains, roads

city, state, state_abbreviation, population

mountain, state, state_abbreviation, height

river, length, [states through which it flows]

road, [states it passes through]

lake, area, [states it is in]


### How to do that? Transform a graph-structure knowledge base to table-like is easy.
1. Read in 'base' file to get all the values
2. structured graph to table
3. analyze logical forms and rewrite
4. data augmentation and train